In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')
    
# csv_to_parquet('./train.csv', 'train')
# csv_to_parquet('./test.csv', 'test')

In [10]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [11]:
train = train.dropna(axis=0)
train

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
13,TRAIN_000013,9,19,615.0,706.0,0,0,MAF,13158,Texas,DEN,11292,Colorado,563.0,SkyWest Airlines Inc.,UA,20304.0,N165SY,Not_Delayed
19,TRAIN_000019,7,14,1907.0,2145.0,0,0,BDL,10529,Connecticut,ATL,10397,Georgia,859.0,Delta Air Lines Inc.,DL,19790.0,N962DN,Not_Delayed
42,TRAIN_000042,10,24,1005.0,1130.0,0,0,ORD,13930,Illinois,STL,15016,Missouri,258.0,Trans States Airlines,UA,20237.0,N835HK,Not_Delayed
59,TRAIN_000059,5,2,1732.0,1935.0,0,0,RIC,14524,Virginia,MCO,13204,Florida,667.0,JetBlue Airways,B6,20409.0,N258JB,Not_Delayed
63,TRAIN_000063,12,28,1820.0,1925.0,0,0,OAK,13796,California,BUR,10800,California,325.0,Southwest Airlines Co.,WN,19393.0,N777QC,Not_Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999943,TRAIN_999943,9,30,1100.0,1155.0,0,0,IND,12339,Indiana,LAS,12889,Nevada,1590.0,Southwest Airlines Co.,WN,19393.0,N8532S,Not_Delayed
999950,TRAIN_999950,11,7,1401.0,1950.0,0,0,PSP,14262,California,ORD,13930,Illinois,1652.0,United Air Lines Inc.,UA,19977.0,N896UA,Not_Delayed
999955,TRAIN_999955,4,12,1310.0,1356.0,0,0,SEA,14747,Washington,PDX,14057,Oregon,129.0,Horizon Air,AS,19687.0,N432QX,Delayed
999985,TRAIN_999985,8,8,1914.0,2039.0,0,0,RDU,14492,North Carolina,JAX,12451,Florida,407.0,Frontier Airlines Inc.,F9,20436.0,N316FR,Not_Delayed


In [12]:
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
#     mode = train[col].mode()[0]
#     train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(train[col].mode()[0])
print('Done.')

qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.
Done.


In [13]:
# train = train.dropna()
# column_number = {}
# for i, column in enumerate(sample_submission.columns):
#     column_number[column] = i
    
# def to_number(x, dic):
#     return dic[x]

# train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
# print('Done.')

In [14]:
# train['Delay'] = le.fit_transform(train['Delay'])

In [15]:
train = train.drop(columns=['ID'])
train = train.reset_index(drop=True)

# train_y = train['Delay_num']
# train_y = train_y.reset_index(drop=True)

test = test.drop(columns=['ID'])

In [17]:
target = 'Delay'
features = [c for c in train.columns if c not in [target]]

In [19]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = KFold(n_splits = 10, random_state = 41, shuffle = True)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

# bundle = [CatBoostClassifier, LGBMClassifier, XGBClassifier]
# for model in bundle:
# model = CatBoostClassifier(**params)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(train, train[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train[features].loc[train_index], train[features].loc[val_index]
        y_train, y_val = train[target][train_index], train[target][val_index]

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = log_loss(y_val, model.predict_proba(X_val))

        val_scores.append(score)

0it [00:00, ?it/s]

0:	learn: 0.6878222	test: 0.6877996	best: 0.6877996 (0)	total: 65.5ms	remaining: 5m 27s
2000:	learn: 0.4282721	test: 0.4355061	best: 0.4355061 (2000)	total: 17.3s	remaining: 25.9s
4000:	learn: 0.4145342	test: 0.4346265	best: 0.4345817 (3881)	total: 35.3s	remaining: 8.8s


1it [00:36, 36.49s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4345817178
bestIteration = 3881

Shrink model to first 3882 iterations.
0:	learn: 0.6878308	test: 0.6877573	best: 0.6877573 (0)	total: 9.58ms	remaining: 47.9s
2000:	learn: 0.4281822	test: 0.4368638	best: 0.4368638 (2000)	total: 17.5s	remaining: 26.2s
4000:	learn: 0.4145818	test: 0.4357239	best: 0.4357239 (4000)	total: 35.1s	remaining: 8.77s


2it [01:16, 38.45s/it]

...

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4364941164
bestIteration = 4382

Shrink model to first 4383 iterations.
0:	learn: 0.6877933	test: 0.6878703	best: 0.6878703 (0)	total: 11.4ms	remaining: 56.9s
2000:	learn: 0.4270324	test: 0.4471126	best: 0.4471126 (2000)	total: 17.9s	remaining: 26.9s


9it [05:48, 39.01s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4462912209
bestIteration = 3682

Shrink model to first 3683 iterations.
0:	learn: 0.6878013	test: 0.6878305	best: 0.6878305 (0)	total: 9.21ms	remaining: 46s
2000:	learn: 0.4276933	test: 0.4419723	best: 0.4419723 (2000)	total: 19s	remaining: 28.4s
4000:	learn: 0.4141680	test: 0.4412495	best: 0.4412383 (3856)	total: 37.4s	remaining: 9.34s


10it [06:26, 38.66s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4412382792
bestIteration = 3856

Shrink model to first 3857 iterations.


In [20]:
val_scores

[0.43458171775239673,
 0.4356648996813724,
 0.4383582894949309,
 0.44565824682683775,
 0.43242458927469923,
 0.4353388641482646,
 0.43785904076377596,
 0.43649411638636254,
 0.44629122094531626,
 0.4412382792435065]

In [21]:
cat_preds = []
for model in models[:10]:
    preds = model.predict_proba(test[features])
    cat_preds.append(preds)

In [22]:
pred = np.array(cat_preds).mean(axis=0)

submission = pd.DataFrame(data=pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('baseline_submission.csv', index=True)